# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [27]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats import weightstats as stests

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


Q1: What test is appropriate for this problem? Does CLT apply?
A1: We will use a two-sample t-test since we are comparing the means of two groups.The CLT will apply because we have a large number of samples (more than 30 ) and each observation is independent from each other. 
Q2: What are the null and alternate hypotheses?
A2: Null hypothesis: Mean of the number of calls for black-sounding applicants received from employer equals to the mean of the number of calls for white-sounding applicants received from empolyer
    Alternative hypothesis: Mean of the number of calls for black-sounding applicants received from employer is not the same as the mean of the number of calls for white-sounding applicants received from empolyer. 

In [20]:
w = data[data.race=='w']
b = data[data.race=='b']
w_count = np.count_nonzero(w)
b_count = np.count_nonzero(b)

w_received = w[w.call == 1]
b_received = b[b.call == 1]

w_received_count = np.count_nonzero(w_received.call)
b_received_count = np.count_nonzero(b_received.call)

w_probability_received = w_received_count/w_count
b_probability_received = b_received_count/b_count

print('Probability of white sounding received call: ', w_probability_received)
print('Probability of black sounding received call: ', b_probability_received)

Probability of white sounding received call:  0.002427660871271991
Probability of black sounding received call:  0.0016215657921916959


In [29]:
# Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
# margin of error ( 95% Confidence )
z = 1.96
mu_w = (1-w_probability_received)*0 + w_probability_received * 1 # Mean of white sounding 
var_w = (1-w_probability_received)*((0-mu_w)**2) + w_probability_received * ((1-mu_w)**2) #variance of white sounding
sigma_w = np.sqrt(var_w) # standard deviation of white sounding

margin_of_error_w = z * (sigma_w/np.sqrt(w_count)) #margin of error of white sounding
print('Margin of Error for White-sounding: ', margin_of_error_w)

mu_b = (1-b_probability_received)*0 + b_probability_received * 1  # Mean of black sounding
var_b = (1-b_probability_received)*((0-mu_b)**2) + b_probability_received * ((1-mu_b)**2) #variance of black sounding
sigma_b = np.sqrt(var_b)  #standard deviation of black sounding

margin_of_error_b = z * (sigma_b/np.sqrt(b_count)) # margin of error of black sounding
print('Margin of Error for Black-sounding: ', margin_of_error_b)

# Confidence interval
# white sounding CI
w_CI_upper = mu_w + margin_of_error_w
w_CI_lower = mu_w - margin_of_error_w

print('The 95% Confidence Interval for white sounding is:' , w_CI_lower, w_CI_upper)

# white sounding CI
b_CI_upper = mu_b + margin_of_error_b
b_CI_lower = mu_b - margin_of_error_b

print('The 95% Confidence Interval for black sounding is:' , b_CI_lower, b_CI_upper)

testResult_job=stests.ztest(w.call,b.call,alternative='two-sided')
pValue_job=testResult_job[1]
print("p-value for z-test is: "+str(pValue_job))


Margin of Error for White-sounding:  0.0003100147503991845
Margin of Error for Black-sounding:  0.00025344773530569146
The 95% Confidence Interval for white sounding is: 0.0021176461208728065 0.0027376756216711757
The 95% Confidence Interval for black sounding is: 0.0013681180568860044 0.0018750135274973874
p-value for z-test is: 3.8767429116085706e-05


Q4.Write a story describing the statistical significance in the context or the original problem.
A4: The p-value of this two-sample z test is much smaller than alpha value of 0.05. We will reject the null hypothesis which is an indication that mean number of calls received for white-sounding and black-sounding does not equal. From the confidence interval, we can find out that white-sounding applicants has a higher chance to receive calls. 
5.Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
A5: Current analysis can only prove that race/name is one factor in callback success. Because we have not analyze the impact of other variables in the dataset. Other information might have a bigger impact on the callback success. We can run an analysis of variance to figure out which factor/factors make the most impact of callback success, then run the hypothesis test on those factor/factors. 